In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q sentence_transformers
!pip install -q umap
!pip install -q umap-learn

In [ ]:
import pandas as pd
import numpy as np
import torch
import itertools
from datasets import Dataset
from scipy import stats
from torch.utils.data import DataLoader
from sklearn import metrics
from collections import Counter
from sentence_transformers import models, SentenceTransformer, losses, evaluation, InputExample

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive')
import eval_utils

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def make_cls_pairs_bin_similarity(df, bin_column, transcript_column):
    train_examples = []
    processed_pairs = set()
    
    for i, row in df.iterrows():
        # Get sample
        sample = row[transcript_column]
        sample_task = row['task']
        sample_bin = row[bin_column]
        
        # Get pairs within the same task
        task_df = df[(df['task'] == sample_task) & (df[transcript_column] != sample)].copy()
        
        for j, pair_row in task_df.iterrows():
            pair_sample = pair_row[transcript_column]
            
            # Check if the pair has already been processed
            if (sample, pair_sample) in processed_pairs or (pair_sample, sample) in processed_pairs:
                continue

            pair_bin = pair_row[bin_column]
            
            # Calculate cosine similarity label based on bins
            similarity_label = 1 if sample_bin == pair_bin else 0
            
            train_examples.append(InputExample(texts=[sample, pair_sample], label=similarity_label))

            # Add the processed pair to the set
            processed_pairs.add((sample, pair_sample))
    
    return train_examples

In [ ]:
df = pd.read_csv("drive/MyDrive/finnish_average.csv")

model_name = "drive/MyDrive/FIN_TASK_MODELS_SIAM/epoch3_split{}"

criterion_column = 'task_completion_mean' #'ta_facets'
bin_column = 'ta_bins' #'ta_bins_r'
folder_name = "FIN_TA_MODELS_SIAM_BIN_TASK"

results_df = pd.DataFrame()
all_true = []
all_samples = []

for e in range(3):
  print("epoch "+str(e))
  all_predictions = []
  pred_before_training = []
  for split in df['split'].unique():
    print("----------------------------")
    print("split {}".format(split))
    
    # make saving path
    model_path = "drive/MyDrive/"+folder_name+"/epoch{}_split{}".format(e,split)
    
    train_df = df[df['split']!=split].reset_index(drop=True)
    test_df = df[df['split']==split].reset_index(drop=True)
    
    if e == 0:
      # add values to the df
      split_true = test_df[criterion_column].tolist()
      all_true+=split_true
      
      split_sample = test_df['sample'].tolist()
      all_samples+=split_sample

      # load untrained model
      model = SentenceTransformer(model_name.format(split), device=device)

      pre_emb_dict = eval_utils.get_embed_dict(df['clean_transcript'].unique().tolist(), model)
      train_df['pre_training_embeds'] = [pre_emb_dict[sent] for sent in train_df['clean_transcript']]
      test_df['pre_training_embeds'] = [pre_emb_dict[sent] for sent in test_df['clean_transcript']]

          
      y_pred_ta = eval_utils.get_bert_n_closest_score(train_df,
                                                      test_df,
                                                      "pre_training_embeds",
                                                      criterion_column)
      pred_before_training+=y_pred_ta
      
    else:
      # load trained in previous epoch
      pre_model_path = "drive/MyDrive/"+folder_name+"/epoch{}_split{}".format(e-1,split)
      model = SentenceTransformer(pre_model_path, device=device)
    
    
    emb_dict = eval_utils.get_embed_dict(train_df['clean_transcript'].unique().tolist(), model)
    train_df['pre_training_embeds'] = [emb_dict[sent] for sent in train_df['clean_transcript']]
    
    train_examples = make_cls_pairs_bin_similarity(train_df, bin_column, "clean_transcript")
    print(len(train_examples))
    #random.shuffle(train_examples)
    #train_examples = train_examples[:1500]

    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

    train_loss = losses.ContrastiveLoss(model=model)
    
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=0)
    model.save(model_path)
    
    emb_dict = eval_utils.get_embed_dict(df['clean_transcript'].unique().tolist(), model.eval())
    train_df['post_training_embeds'] = [emb_dict[sent] for sent in train_df['clean_transcript']]
    test_df['post_training_embeds'] = [emb_dict[sent] for sent in test_df['clean_transcript']]
    df['post_training_embeds'] = [emb_dict[sent] for sent in df['clean_transcript']]
    
    y_pred_ta = eval_utils.get_bert_n_closest_score(train_df,
                                                    test_df,
                                                    "post_training_embeds",
                                                    criterion_column)

    all_predictions+=y_pred_ta

  
  if e==0:
    results_df['samples']=all_samples
    results_df['true']=all_true
    results_df['pre']=pred_before_training
    results_df['epoch0']=all_predictions
  else:
    results_df['epoch'+str(e)]=all_predictions
  
  print("POST TRAINING 1NN")
  eval_utils.evaluate_cls(all_true, all_predictions)
  print('----------')
  eval_utils.evaluate_reg(all_true, all_predictions, "sbert 1nn epoch {}".format(e))
  print('----------')
  eval_utils.compute_task_scores(df, 'task','post_training_embeds')
  print('----------')
  eval_utils.compute_bin_scores(df, 'post_training_embeds', bin_column)
  eval_utils.plot_n_random_tasks(df, 'task', 'post_training_embeds', n=10)
  eval_utils.plot_subtask(df, 'task', 1, 'post_training_embeds', criterion_column)
  #-----------------------------------------

  results_df.to_csv("drive/MyDrive/"+folder_name+"/results_cls.csv", index=False)